# Introduction to Pandas Continuation

In [2]:
import pandas as pd
import numpy as np
import yfinance
AAPL = yfinance.download(["AAPL"],auto_adjust=False,period="5Y")
AAPL=pd.DataFrame(AAPL.values,AAPL.index,columns=AAPL.columns.get_level_values('Price').values)

[*********************100%***********************]  1 of 1 completed


# Rolling windows

### In time-series it is useful to apply a function on observation windows of size $n$. Basically we want to use the `apply()` logic on $M$ grouped observations i.e. assuming we have a vector of observations $X=(X_1,...,X_N)$ then we want 
$$\text{Rolling}(X,f(\cdot):\mathbb{R}^M\mapsto \mathbb{R},M)=(Y_1,...,Y_{N-M+1})$$
where
$Y_i=f(X_{i-M+1},...,X_i)$.

We can do so using ```dataframe.rolling(M)```. The this is followed by the function we want to apply to each rolling window; tipycal choices are ```mean,std,min,max```. For more details and additional arguments see (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)


In [9]:
# Lets compute the 10 day moving average
AAPL.Close.rolling(10).mean()

Date
2020-11-06           NaN
2020-11-09           NaN
2020-11-10           NaN
2020-11-11           NaN
2020-11-12           NaN
                 ...    
2025-10-30    263.705998
2025-10-31    265.513998
2025-11-03    266.194998
2025-11-04    266.922000
2025-11-05    268.091000
Name: Close, Length: 1255, dtype: float64

In [10]:
# Lets compute the 10 day rolling minimum
AAPL.Close.rolling(10).min()

Date
2020-11-06           NaN
2020-11-09           NaN
2020-11-10           NaN
2020-11-11           NaN
2020-11-12           NaN
                 ...    
2025-10-30    252.289993
2025-10-31    258.450012
2025-11-03    258.450012
2025-11-04    258.450012
2025-11-05    259.579987
Name: Close, Length: 1255, dtype: float64

In [11]:
# Lets compute the 10 day rolling std
AAPL.Close.rolling(10).std()

Date
2020-11-06         NaN
2020-11-09         NaN
2020-11-10         NaN
2020-11-11         NaN
2020-11-12         NaN
                ...   
2025-10-30    6.036812
2025-10-31    4.823356
2025-11-03    4.790377
2025-11-04    4.764422
2025-11-05    3.789062
Name: Close, Length: 1255, dtype: float64

#### You can mix ```rolling()``` with ```apply()``` to use custom functions on a rolling window basis 

In [29]:
AAPL.Close.rolling(10).apply(lambda x: x.max() - x.min(),raw=True)

Date
2020-11-06          NaN
2020-11-09          NaN
2020-11-10          NaN
2020-11-11          NaN
2020-11-12          NaN
                ...    
2025-10-30    19.110001
2025-10-31    12.949982
2025-11-03    12.949982
2025-11-04    12.949982
2025-11-05    11.820007
Name: Close, Length: 1255, dtype: float64

In [30]:
AAPL.Close.rolling(3).apply(lambda x: x.iat[-1]/x.iat[0])

Date
2020-11-06         NaN
2020-11-09         NaN
2020-11-10    0.977083
2020-11-11    1.027252
2020-11-12    1.027938
                ...   
2025-10-30    1.008922
2025-10-31    1.002484
2025-11-03    0.991341
2025-11-04    0.998780
2025-11-05    1.004051
Name: Close, Length: 1255, dtype: float64

### Note that if you want to use raw=True for performance you will need to treat the variables as numpy arrays

In [31]:
#this will fail
AAPL.Close.rolling(3).apply(lambda x: x.iat[-1]/x.iat[0],raw=True)

AttributeError: 'numpy.ndarray' object has no attribute 'iat'

In [32]:
AAPL.Close.rolling(3).apply(lambda x: x[-1]/x[0],raw=True)

Date
2020-11-06         NaN
2020-11-09         NaN
2020-11-10    0.977083
2020-11-11    1.027252
2020-11-12    1.027938
                ...   
2025-10-30    1.008922
2025-10-31    1.002484
2025-11-03    0.991341
2025-11-04    0.998780
2025-11-05    1.004051
Name: Close, Length: 1255, dtype: float64

In [35]:
%timeit AAPL.Close.rolling(3).apply(lambda x: x[-1]/x[0],raw=True)
%timeit AAPL.Close.rolling(3).apply(lambda x: x.iat[-1]/x.iat[0])
%timeit AAPL.Close/AAPL.Close.shift(2)

552 μs ± 5.32 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
22.7 ms ± 197 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
67.1 μs ± 2.91 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### Vectorized solutions are always fastest

In [37]:
## Last one is equivalent to
AAPL.Close/AAPL.Close.shift(2)

Date
2020-11-06         NaN
2020-11-09         NaN
2020-11-10    0.977083
2020-11-11    1.027252
2020-11-12    1.027938
                ...   
2025-10-30    1.008922
2025-10-31    1.002484
2025-11-03    0.991341
2025-11-04    0.998780
2025-11-05    1.004051
Name: Close, Length: 1255, dtype: float64

---
# Applying custom rolling logic to multiple columns
#### If we apply a rolling window function to a dataframe it will return the same structured dataframe with the rolling logic applied columnwise, unfortunately if we want to apply a logic that involves taking values over multiple columns this method is not enough.

#### Without getting into additional libraries, the way to go in this case is to use the numpy native ```np.lib.stride_tricks.sliding_window_view()``` which allows us to create rolling windows in tensors, not just vectors

#### We can use the combination of ```window_shape``` and ```axis``` to decide along which of the axis we want to create a rolling window

In [93]:
np.lib.stride_tricks.sliding_window_view(AAPL.values, window_shape=10,axis=0).shape

(1246, 6, 10)

#### The following is equivalent to finding the rolling 10 day mean for each column

In [94]:
np.lib.stride_tricks.sliding_window_view(AAPL.values, 10,axis=0).mean(axis=2)

array([[1.15495157e+02, 1.18530000e+02, 1.19914998e+02, 1.17111000e+02,
        1.18529001e+02, 1.01992490e+08],
       [1.15363611e+02, 1.18395000e+02, 1.19871998e+02, 1.17227000e+02,
        1.18561001e+02, 9.79071300e+07],
       [1.15122935e+02, 1.18148000e+02, 1.19434998e+02, 1.16997000e+02,
        1.18229001e+02, 9.52515300e+07],
       ...,
       [2.66194998e+02, 2.66194998e+02, 2.68562006e+02, 2.63824997e+02,
        2.66820999e+02, 5.06477700e+07],
       [2.66922000e+02, 2.66922000e+02, 2.69182004e+02, 2.64403998e+02,
        2.67465997e+02, 5.09056600e+07],
       [2.68091000e+02, 2.68091000e+02, 2.70067004e+02, 2.65553998e+02,
        2.68059998e+02, 5.04420587e+07]], shape=(1246, 6))

In [97]:
AAPL.rolling(10).mean().dropna()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-11-19,115.495157,118.530000,119.914998,117.111000,118.529001,101992490.0
2020-11-20,115.363611,118.395000,119.871998,117.227000,118.561001,97907130.0
2020-11-23,115.122935,118.148000,119.434998,116.997000,118.229001,95251530.0
2020-11-24,115.044983,118.067999,119.260999,116.843000,118.065001,92836610.0
2020-11-25,114.707840,117.721999,118.972999,116.716000,117.901001,89257030.0
...,...,...,...,...,...,...
2025-10-30,263.705998,263.705998,265.521005,260.573997,262.470999,50974600.0
2025-10-31,265.513998,265.513998,267.915005,262.762997,265.367998,54676610.0
2025-11-03,266.194998,266.194998,268.562006,263.824997,266.820999,50647770.0


In [98]:
pd.DataFrame(data=np.lib.stride_tricks.sliding_window_view(AAPL.values, 10,axis=0).mean(axis=2))

,0,1,2,3,4,5
0,115.495157,118.530000,119.914998,117.111000,118.529001,101992490.0
1,115.363611,118.395000,119.871998,117.227000,118.561001,97907130.0
2,115.122935,118.148000,119.434998,116.997000,118.229001,95251530.0
3,115.044983,118.067999,119.260999,116.843000,118.065001,92836610.0
4,114.707840,117.721999,118.972999,116.716000,117.901001,89257030.0
...,...,...,...,...,...,...
1241,263.705998,263.705998,265.521005,260.573997,262.470999,50974600.0
1242,265.513998,265.513998,267.915005,262.762997,265.367998,54676610.0
1243,266.194998,266.194998,268.562006,263.824997,266.820999,50647770.0
1244,266.922000,266.922000,269.182004,264.403998,267.465997,50905660.0


#### We can tackle more complex logic, like computing the min-max average spread over a 10 day rolling window 

#### To do so, the equivalent of ```apply``` for numpy is ```np.apply_along_axis```, which is necessary due to the tensor algebra structure of numpy. ```apply``` only works on 1D vectors, and we should think of ```np.apply_along_axis``` as the extension to apply over arbitrary axis


In [136]:
np.random.seed(0)
np.random.normal(0,1,(5,2))

array([[ 1.76405235,  0.40015721],
       [ 0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788],
       [ 0.95008842, -0.15135721],
       [-0.10321885,  0.4105985 ]])

In [137]:
np.random.seed(0)
np.apply_along_axis(lambda x: x[0]-x[1], axis=1, arr=np.random.normal(0,1,(5,2)))

array([ 1.36389514, -1.26215522,  2.84483587,  1.10144563, -0.51381735])

In [138]:
np.apply_along_axis(lambda x: x[0]-x[1], axis=1,arr=np.lib.stride_tricks.sliding_window_view(AAPL.loc[:,["High","Low"]], 10,axis=0)).mean(axis=1)

array([2.8039978 , 2.64499741, 2.4379982 , ..., 4.73700867, 4.77800598,
       4.51300659], shape=(1246,))

In [132]:
(AAPL.High-AAPL.Low).rolling(10).mean().dropna()

Date
2020-11-19    2.803998
2020-11-20    2.644997
2020-11-23    2.437998
2020-11-24    2.417999
2020-11-25    2.256999
                ...   
2025-10-30    4.947008
2025-10-31    5.152008
2025-11-03    4.737009
2025-11-04    4.778006
2025-11-05    4.513007
Length: 1246, dtype: float64

#### This is a good example of why numpy is richer than pandas, and is useful to be be able to do the same thing different ways. 

#### One **important** thing to note is that numpy windows are not memory consuming as they reference the original data:

In [144]:
import sys
print(f" Size of Numpy rolling window object: { sys.getsizeof(np.lib.stride_tricks.sliding_window_view(AAPL.loc[:,["High","Low"]], 10,axis=0)):.0f} bytes")

 Size of Numpy rolling window object: 144 bytes


---
# Join/Merge/Concatenate dataframes and series

## 1 Join and Merge

### Now that we have a clear view on basic data manipulation, we can ask ourselves how can we merge data from different Stocks?

In [145]:
list_of_symbols=['aapl','amzn','dis','msft','spy']

dict_of_df={}
for symbol in list_of_symbols:
    aux=yfinance.download([symbol],auto_adjust=False,period="5Y") 
    dict_of_df[symbol]=pd.DataFrame(aux.values,index=aux.index,columns=aux.columns.get_level_values('Price').values)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [146]:
dict_of_df['amzn']

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-11-06,165.568497,165.568497,166.100006,161.600006,165.231995,92946000.0
2020-11-09,157.186996,157.186996,164.449997,155.605499,161.551498,143808000.0
2020-11-10,151.751007,151.751007,155.699997,150.973999,154.751007,131820000.0
2020-11-11,156.869507,156.869507,156.957504,152.500000,153.089005,87338000.0
2020-11-12,155.514008,155.514008,158.794006,154.302505,157.997498,87240000.0
...,...,...,...,...,...,...
2025-10-30,222.860001,222.860001,228.440002,222.750000,227.059998,102252900.0
2025-10-31,244.220001,244.220001,250.500000,243.979996,250.100006,166340800.0
2025-11-03,254.000000,254.000000,258.600006,252.899994,255.360001,95997800.0


In [147]:
dict_of_df['aapl']

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-11-06,115.651062,118.690002,119.199997,116.129997,118.320000,114457900.0
2020-11-09,113.341743,116.320000,121.989998,116.050003,120.500000,154515300.0
2020-11-10,113.000702,115.970001,117.589996,114.129997,115.550003,138023400.0
2020-11-11,116.430573,119.489998,119.629997,116.440002,117.190002,112295000.0
2020-11-12,116.157745,119.209999,120.529999,118.570000,119.620003,103162300.0
...,...,...,...,...,...,...
2025-10-30,271.399994,271.399994,274.140015,268.480011,271.989990,69886500.0
2025-10-31,270.369995,270.369995,277.320007,269.160004,276.989990,86167100.0
2025-11-03,269.049988,269.049988,270.850006,266.250000,270.420013,50194600.0


### We can merge two dataframes by index using ```merge```. By setting ```how='inner'``` we make sure that only intersecting indices will be selected and likewise setting ```left_index=True```, ```right_index=True``` we make sure that both indices are being considered 

In [148]:
%timeit merged_df1=dict_of_df['aapl'].merge(dict_of_df['amzn'],how='inner',left_index=True, right_index=True,suffixes=('_aapl','_amzn'))

326 μs ± 13.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [149]:
merged_df1=dict_of_df['aapl'].merge(dict_of_df['amzn'],how='inner',left_index=True, right_index=True,suffixes=('_aapl','_amzn'))
merged_df1.head()

,Adj Close_aapl,Close_aapl,High_aapl,Low_aapl,Open_aapl,Volume_aapl,Adj Close_amzn,Close_amzn,High_amzn,Low_amzn,Open_amzn,Volume_amzn
Date,,,,,,,,,,,,
2020-11-06,115.651062,118.690002,119.199997,116.129997,118.320000,114457900.0,165.568497,165.568497,166.100006,161.600006,165.231995,92946000.0
2020-11-09,113.341743,116.320000,121.989998,116.050003,120.500000,154515300.0,157.186996,157.186996,164.449997,155.605499,161.551498,143808000.0
2020-11-10,113.000702,115.970001,117.589996,114.129997,115.550003,138023400.0,151.751007,151.751007,155.699997,150.973999,154.751007,131820000.0
2020-11-11,116.430573,119.489998,119.629997,116.440002,117.190002,112295000.0,156.869507,156.869507,156.957504,152.500000,153.089005,87338000.0
2020-11-12,116.157745,119.209999,120.529999,118.570000,119.620003,103162300.0,155.514008,155.514008,158.794006,154.302505,157.997498,87240000.0


### Likewise we can perform the same operation using  ```join```

In [150]:
%timeit merged_df2=dict_of_df['aapl'].join(dict_of_df['amzn'],how='inner',lsuffix='_aapl',rsuffix='_amzn')

340 μs ± 26.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [151]:
merged_df2=dict_of_df['aapl'].join(dict_of_df['amzn'],how='inner',lsuffix='_aapl',rsuffix='_amz')
merged_df2.head()

,Adj Close_aapl,Close_aapl,High_aapl,Low_aapl,Open_aapl,Volume_aapl,Adj Close_amz,Close_amz,High_amz,Low_amz,Open_amz,Volume_amz
Date,,,,,,,,,,,,
2020-11-06,115.651062,118.690002,119.199997,116.129997,118.320000,114457900.0,165.568497,165.568497,166.100006,161.600006,165.231995,92946000.0
2020-11-09,113.341743,116.320000,121.989998,116.050003,120.500000,154515300.0,157.186996,157.186996,164.449997,155.605499,161.551498,143808000.0
2020-11-10,113.000702,115.970001,117.589996,114.129997,115.550003,138023400.0,151.751007,151.751007,155.699997,150.973999,154.751007,131820000.0
2020-11-11,116.430573,119.489998,119.629997,116.440002,117.190002,112295000.0,156.869507,156.869507,156.957504,152.500000,153.089005,87338000.0
2020-11-12,116.157745,119.209999,120.529999,118.570000,119.620003,103162300.0,155.514008,155.514008,158.794006,154.302505,157.997498,87240000.0


### ***Remark***: the difference between ```merge``` and ```join``` is that ```merge``` allows for more flexibility allowing to merge by column values as well. In general, merge performance tends to be better at the cost of providing more arguments

---
## 2. Adding rows to an existing dataframe
### ```pd.concat``` allows to add additional rows to an existing dataframe

In [152]:
dataframe=pd.DataFrame({"Open": 100,"High": 100,"Low": 100,"Close": 100 },index=[0])
print(dataframe)
AAPL=pd.concat([AAPL,dataframe],ignore_index=True)

   Open  High  Low  Close
0   100   100  100    100


In [153]:
AAPL

,Adj Close,Close,High,Low,Open,Volume
0,115.651062,118.690002,119.199997,116.129997,118.320000,114457900.0
1,113.341728,116.320000,121.989998,116.050003,120.500000,154515300.0
2,113.000702,115.970001,117.589996,114.129997,115.550003,138023400.0
3,116.430588,119.489998,119.629997,116.440002,117.190002,112295000.0
4,116.157745,119.209999,120.529999,118.570000,119.620003,103162300.0
...,...,...,...,...,...,...
1251,270.369995,270.369995,277.320007,269.160004,276.989990,86167100.0
1252,269.049988,269.049988,270.850006,266.250000,270.420013,50194600.0
1253,270.040009,270.040009,271.489990,267.619995,268.329987,49274800.0
1254,270.140015,270.140015,271.700012,266.929993,268.589996,40379287.0


In [154]:
pd.concat([dict_of_df['aapl'],dict_of_df['amzn']])

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-11-06,115.651062,118.690002,119.199997,116.129997,118.320000,114457900.0
2020-11-09,113.341743,116.320000,121.989998,116.050003,120.500000,154515300.0
2020-11-10,113.000702,115.970001,117.589996,114.129997,115.550003,138023400.0
2020-11-11,116.430573,119.489998,119.629997,116.440002,117.190002,112295000.0
2020-11-12,116.157745,119.209999,120.529999,118.570000,119.620003,103162300.0
...,...,...,...,...,...,...
2025-10-30,222.860001,222.860001,228.440002,222.750000,227.059998,102252900.0
2025-10-31,244.220001,244.220001,250.500000,243.979996,250.100006,166340800.0
2025-11-03,254.000000,254.000000,258.600006,252.899994,255.360001,95997800.0


---
## 3. Concat dictionaries and Multi_indexing
### Concatenate allows to merge multiple df at once, but will create a multi-index/multi-column data frame

In [155]:
df1=pd.concat(dict_of_df,axis=1)
df1

aapl                                                  \
             Adj Close       Close        High         Low        Open   
Date                                                                     
2020-11-06  115.651062  118.690002  119.199997  116.129997  118.320000   
2020-11-09  113.341743  116.320000  121.989998  116.050003  120.500000   
2020-11-10  113.000702  115.970001  117.589996  114.129997  115.550003   
2020-11-11  116.430573  119.489998  119.629997  116.440002  117.190002   
2020-11-12  116.157745  119.209999  120.529999  118.570000  119.620003   
...                ...         ...         ...         ...         ...   
2025-10-30  271.399994  271.399994  274.140015  268.480011  271.989990   
2025-10-31  270.369995  270.369995  277.320007  269.160004  276.989990   
2025-11-03  269.049988  269.049988  270.850006  266.250000  270.420013   
2025-11-04  270.040009  270.040009  271.489990  267.619995  268.329987   
2025-11-05  270.140015  270.140015  271.700012  266.929993  268.589996   

                               amzn                                      ...  \
                 Volume   Adj Close       Close        High         Low  ...   
Date                                                                     ...   
2020-11-06  114457900.0  165.568497  165.568497  166.100006  161.600006  ...   
2020-11-09  154515300.0  157.186996  157.186996  164.449997  155.605499  ...   
2020-11-10  138023400.0  151.751007  151.751007  155.699997  150.973999  ...   
2020-11-11  112295000.0  156.869507  156.869507  156.957504  152.500000  ...   
2020-11-12  103162300.0  155.514008  155.514008  158.794006  154.302505  ...   
...                 ...         ...         ...         ...         ...  ...   
2025-10-30   69886500.0  222.860001  222.860001  228.440002  222.750000  ...   
2025-10-31   86167100.0  244.220001  244.220001  250.500000  243.979996  ...   
2025-11-03   50194600.0  254.000000  254.000000  258.600006  252.899994  ...   
2025-11-04   49274800.0  249.320007  249.320007  257.010010  248.660004  ...   
2025-11-05   40379287.0  250.199997  250.199997  251.000000  246.160004  ...   

                  msft                                             spy  \
                  High         Low        Open      Volume   Adj Close   
Date                                                                     
2020-11-06  224.360001  218.029999  222.259995  25231900.0  326.440399   
2020-11-09  228.119995  217.880005  224.440002  44395000.0  330.542328   
2020-11-10  216.500000  209.720001  214.500000  44045100.0  330.057617   
2020-11-11  218.039993  212.199997  212.389999  29440800.0  332.509491   
2020-11-12  219.110001  214.460007  217.210007  21593900.0  329.283752   
...                ...         ...         ...         ...         ...   
2025-10-30  534.969971  522.119995  530.479980  41023100.0  679.830017   
2025-10-31  529.320007  515.099976  528.880005  34006400.0  682.059998   
2025-11-03  524.960022  514.590027  519.809998  22374700.0  683.340027   
2025-11-04  515.549988  507.839996  511.760010  20958700.0  675.239990   
2025-11-05  514.830017  506.575012  513.320007  22927843.0  677.580017   

                                                                         
                 Close        High         Low        Open       Volume  
Date                                                                     
2020-11-06  350.160004  351.510010  347.649994  349.929993   74973000.0  
2020-11-09  354.559998  364.380005  354.059998  363.970001  172304200.0  
2020-11-10  354.040009  355.179993  350.510010  353.489990   85552000.0  
2020-11-11  356.670013  357.559998  355.059998  356.399994   58649000.0  
2020-11-12  353.209991  356.720001  351.260010  355.579987   67546200.0  
...                ...         ...         ...         ...          ...  
2025-10-30  679.830017  685.940002  679.830017  683.900024   76335800.0  
2025-10-31  682.059998  685.080017  679.239990  685.039978   87164100.0  
2025-11-0

In [156]:
df2=pd.concat(dict_of_df,axis=0)
df2

Adj Close       Close        High         Low        Open  \
     Date                                                                     
aapl 2020-11-06  115.651062  118.690002  119.199997  116.129997  118.320000   
     2020-11-09  113.341743  116.320000  121.989998  116.050003  120.500000   
     2020-11-10  113.000702  115.970001  117.589996  114.129997  115.550003   
     2020-11-11  116.430573  119.489998  119.629997  116.440002  117.190002   
     2020-11-12  116.157745  119.209999  120.529999  118.570000  119.620003   
...                     ...         ...         ...         ...         ...   
spy  2025-10-30  679.830017  679.830017  685.940002  679.830017  683.900024   
     2025-10-31  682.059998  682.059998  685.080017  679.239990  685.039978   
     2025-11-03  683.340027  683.340027  685.799988  679.940002  685.669983   
     2025-11-04  675.239990  675.239990  679.960022  674.580017  676.109985   
     2025-11-05  677.580017  677.580017  680.859985  674.169983  674.979980   

                      Volume  
     Date                     
aapl 2020-11-06  114457900.0  
     2020-11-09  154515300.0  
     2020-11-10  138023400.0  
     2020-11-11  112295000.0  
     2020-11-12  103162300.0  
...                      ...  
spy  2025-10-30   76335800.0  
     2025-10-31   87164100.0  
     2025-11-03   57315000.0  
     2025-11-04   78427000.0  
     2025-11-05   72773754.0  

[6275 rows x 6 columns]

### This introduces multi_indexing e.g. indices or columns that are 2D arrays or in general n dimensional arrays

In [157]:
df1.columns

MultiIndex([('aapl', 'Adj Close'),
            ('aapl',     'Close'),
            ('aapl',      'High'),
            ('aapl',       'Low'),
            ('aapl',      'Open'),
            ('aapl',    'Volume'),
            ('amzn', 'Adj Close'),
            ('amzn',     'Close'),
            ('amzn',      'High'),
            ('amzn',       'Low'),
            ('amzn',      'Open'),
            ('amzn',    'Volume'),
            ( 'dis', 'Adj Close'),
            ( 'dis',     'Close'),
            ( 'dis',      'High'),
            ( 'dis',       'Low'),
            ( 'dis',      'Open'),
            ( 'dis',    'Volume'),
            ('msft', 'Adj Close'),
            ('msft',     'Close'),
            ('msft',      'High'),
            ('msft',       'Low'),
            ('msft',      'Open'),
            ('msft',    'Volume'),
            ( 'spy', 'Adj Close'),
            ( 'spy',     'Close'),
            ( 'spy',      'High'),
            ( 'spy',       'Low'),
            ( 'spy',

### The good news is that we can efficiently use slicing with  ```loc```, see examples below

In [159]:
df1.loc[:,('msft','Close')]#retrieve all rows for msft and close

Date
2020-11-06    223.720001
2020-11-09    218.389999
2020-11-10    211.009995
2020-11-11    216.550003
2020-11-12    215.440002
                 ...    
2025-10-30    525.760010
2025-10-31    517.809998
2025-11-03    517.030029
2025-11-04    514.330017
2025-11-05    507.160004
Name: (msft, Close), Length: 1255, dtype: float64

In [160]:
df1.loc[:,('msft')]#retrieve all rows and columns for msft and close

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2020-11-06,214.508438,223.720001,224.360001,218.029999,222.259995,25231900.0
2020-11-09,209.397858,218.389999,228.119995,217.880005,224.440002,44395000.0
2020-11-10,202.321762,211.009995,216.500000,209.720001,214.500000,44045100.0
2020-11-11,207.633652,216.550003,218.039993,212.199997,212.389999,29440800.0
2020-11-12,206.569336,215.440002,219.110001,214.460007,217.210007,21593900.0
...,...,...,...,...,...,...
2025-10-30,525.760010,525.760010,534.969971,522.119995,530.479980,41023100.0
2025-10-31,517.809998,517.809998,529.320007,515.099976,528.880005,34006400.0
2025-11-03,517.030029,517.030029,524.960022,514.590027,519.809998,22374700.0


### If we want to select all elements at a given index level, we can use ```slice(None)``` at that level instead of the more intuitive ```:``` that we are used to

In [249]:
df1.loc[:,(slice(None),"Close")]

,aapl,amzn,dis,msft,spy
,Close,Close,Close,Close,Close
Date,,,,,
2020-10-30,108.860001,151.807495,121.250000,202.470001,326.540009
2020-11-02,108.769997,150.223999,120.129997,202.330002,330.200012
2020-11-03,110.440002,152.420502,124.019997,206.429993,336.029999
2020-11-04,114.949997,162.057999,125.070000,216.389999,343.540009
2020-11-05,119.029999,166.100006,126.959999,223.289993,350.239990
...,...,...,...,...,...
2025-10-23,259.579987,221.089996,113.029999,520.559998,671.760010
2025-10-24,262.820007,224.210007,111.680000,523.609985,677.250000


### This way we can nicely obtain all close time series for many symbols

In [161]:
df1.loc[:,(slice(None),"Close")].index

DatetimeIndex(['2020-11-06', '2020-11-09', '2020-11-10', '2020-11-11',
               '2020-11-12', '2020-11-13', '2020-11-16', '2020-11-17',
               '2020-11-18', '2020-11-19',
               ...
               '2025-10-23', '2025-10-24', '2025-10-27', '2025-10-28',
               '2025-10-29', '2025-10-30', '2025-10-31', '2025-11-03',
               '2025-11-04', '2025-11-05'],
              dtype='datetime64[ns]', name='Date', length=1255, freq=None)

---
# Creating a Multi-Index and performing Groupby operations


### Groupby allows to index data using different columns or compute summary statistics for groups wihin the data. Let's look at some option data


In [162]:
import numpy as np
import pandas as pd

In [163]:
option_data=pd.read_csv('AMZN_options.csv')
option_data

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date,option_type
0,AMZN201106C01820000,2020-11-02 18:28:21,1820.0,1148.70,1139.35,1143.20,-62.51001,-5.160956,15.0,4.0,0.000010,True,REGULAR,USD,2020-11-06,C
1,AMZN201106C01830000,2020-11-02 18:28:21,1830.0,1146.25,1123.35,1127.75,-56.18994,-4.672994,1.0,2.0,0.000010,True,REGULAR,USD,2020-11-06,C
2,AMZN201106C01840000,2020-11-02 18:28:21,1840.0,1128.45,1116.55,1120.70,-62.75000,-5.267798,1.0,2.0,0.000010,True,REGULAR,USD,2020-11-06,C
3,AMZN201106C01850000,2020-11-02 18:28:21,1850.0,1126.00,1108.85,1112.60,-56.47998,-4.776401,1.0,3.0,0.000010,True,REGULAR,USD,2020-11-06,C
4,AMZN201106C01870000,2020-11-02 14:34:36,1870.0,1201.10,1086.10,1092.25,-17.23999,-1.415039,2.0,2.0,0.000010,True,REGULAR,USD,2020-11-06,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6032,AMZN230120P03600000,2020-10-28 16:43:01,3600.0,980.00,1061.70,1078.45,0.00000,0.000000,1.0,2.0,0.375985,True,REGULAR,USD,2023-01-20,P
6033,AMZN230120P03900000,2020-09-30 15:21:16,3900.0,1166.30,1227.55,1245.00,0.00000,0.000000,1.0,0.0,0.340918,True,REGULAR,USD,2023-01-20,P
6034,AMZN230120P04000000,2020-10-21 13:47:58,4000.0,1247.25,1359.95,1376.50,0.00000,0.000000,55.0,59.0,0.371846,True,REGULAR,USD,2023-01-20,P
6035,AMZN230120P04800000,2020-10-14 19:58:31,4800.0,1795.00,2017.40,2033.00,0.00000,0.000000,5.0,4.0,0.363714,True,REGULAR,USD,2023-01-20,P


### This dataframe contains multiple option data. As you may already now each option is uniquely defined by "expiration_date" and "option_type". We can create a custom multi-index in our dataframe by using the ```dataframe.set_index(["column1","column2",...])```

In [164]:
indexed_option_data=option_data.set_index(["expiration_date","option_type"])
indexed_option_data

contractSymbol        lastTradeDate  strike  \
expiration_date option_type                                                     
2020-11-06      C            AMZN201106C01820000  2020-11-02 18:28:21  1820.0   
                C            AMZN201106C01830000  2020-11-02 18:28:21  1830.0   
                C            AMZN201106C01840000  2020-11-02 18:28:21  1840.0   
                C            AMZN201106C01850000  2020-11-02 18:28:21  1850.0   
                C            AMZN201106C01870000  2020-11-02 14:34:36  1870.0   
...                                          ...                  ...     ...   
2023-01-20      P            AMZN230120P03600000  2020-10-28 16:43:01  3600.0   
                P            AMZN230120P03900000  2020-09-30 15:21:16  3900.0   
                P            AMZN230120P04000000  2020-10-21 13:47:58  4000.0   
                P            AMZN230120P04800000  2020-10-14 19:58:31  4800.0   
                P            AMZN230120P05000000  2020-10-22 18:55:34  5000.0   

                             lastPrice      bid      ask    change  \
expiration_date option_type                                          
2020-11-06      C              1148.70  1139.35  1143.20 -62.51001   
                C              1146.25  1123.35  1127.75 -56.18994   
                C              1128.45  1116.55  1120.70 -62.75000   
                C              1126.00  1108.85  1112.60 -56.47998   
                C              1201.10  1086.10  1092.25 -17.23999   
...                                ...      ...      ...       ...   
2023-01-20      P               980.00  1061.70  1078.45   0.00000   
                P              1166.30  1227.55  1245.00   0.00000   
                P              1247.25  1359.95  1376.50   0.00000   
                P              1795.00  2017.40  2033.00   0.00000   
                P              2051.20  2193.50  2203.00   0.00000   

                             percentChange  volume  openInterest  \
expiration_date option_type                                        
2020-11-06      C                -5.160956    15.0           4.0   
                C                -4.672994     1.0           2.0   
                C                -5.267798     1.0           2.0   
                C                -4.776401     1.0           3.0   
                C                -1.415039     2.0           2.0   
...                                    ...     ...           ...   
2023-01-20      P                 0.000000     1.0           2.0   
                P                 0.000000     1.0           0.0   
                P                 0.000000    55.0          59.0   
                P                 0.000000     5.0           4.0   
                P                 0.000000     1.0           1.0   

                             impliedVolatility  inTheMoney contractSize  \
expiration_date option_type                                               
2020-11-06      C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
...                                        ...         ...          ...   
2023-01-20      P                     0.375985        True      REGULAR   
                P                     0.340918        True      REGULAR   
                P                     0.371846        True      REGULAR   
                P                     0.363714        True      REGULAR   
                P                     0.358921        True      REGULAR   

                            currency  
expiration_date option_type           
2020-11-06      C                USD  
                C                USD  
                C                USD  
             

In [165]:
indexed_option_data.loc[("2020-11-06",),:]##retrieve all options for a given expiration

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
option_type,,,,,,,,,,,,,,
C,AMZN201106C01820000,2020-11-02 18:28:21,1820.0,1148.70,1139.35,1143.20,-62.51001,-5.160956,15.0,4.0,0.000010,True,REGULAR,USD
C,AMZN201106C01830000,2020-11-02 18:28:21,1830.0,1146.25,1123.35,1127.75,-56.18994,-4.672994,1.0,2.0,0.000010,True,REGULAR,USD
C,AMZN201106C01840000,2020-11-02 18:28:21,1840.0,1128.45,1116.55,1120.70,-62.75000,-5.267798,1.0,2.0,0.000010,True,REGULAR,USD
C,AMZN201106C01850000,2020-11-02 18:28:21,1850.0,1126.00,1108.85,1112.60,-56.47998,-4.776401,1.0,3.0,0.000010,True,REGULAR,USD
C,AMZN201106C01870000,2020-11-02 14:34:36,1870.0,1201.10,1086.10,1092.25,-17.23999,-1.415039,2.0,2.0,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P,AMZN201106P04550000,2020-11-02 15:27:12,4550.0,1540.00,1587.50,1591.95,246.44995,19.052217,1.0,1.0,2.030950,True,REGULAR,USD
P,AMZN201106P04650000,2020-10-30 15:11:44,4650.0,1565.65,1686.55,1691.95,0.00000,0.000000,1.0,0.0,2.097234,True,REGULAR,USD
P,AMZN201106P04700000,2020-10-30 17:11:39,4700.0,1661.40,1737.35,1742.15,0.00000,0.000000,4.0,2.0,2.151738,True,REGULAR,USD


In [166]:
indexed_option_data.loc[("2020-11-06","C"),:]#retrieve all Call options for a given expiration

contractSymbol        lastTradeDate  strike  \
expiration_date option_type                                                     
2020-11-06      C            AMZN201106C01820000  2020-11-02 18:28:21  1820.0   
                C            AMZN201106C01830000  2020-11-02 18:28:21  1830.0   
                C            AMZN201106C01840000  2020-11-02 18:28:21  1840.0   
                C            AMZN201106C01850000  2020-11-02 18:28:21  1850.0   
                C            AMZN201106C01870000  2020-11-02 14:34:36  1870.0   
...                                          ...                  ...     ...   
                C            AMZN201106C04650000  2020-11-02 15:54:50  4650.0   
                C            AMZN201106C04700000  2020-10-30 19:55:10  4700.0   
                C            AMZN201106C04750000  2020-11-02 17:32:06  4750.0   
                C            AMZN201106C04800000  2020-10-30 18:56:49  4800.0   
                C            AMZN201106C04850000  2020-11-02 19:06:05  4850.0   

                             lastPrice      bid      ask    change  \
expiration_date option_type                                          
2020-11-06      C              1148.70  1139.35  1143.20 -62.51001   
                C              1146.25  1123.35  1127.75 -56.18994   
                C              1128.45  1116.55  1120.70 -62.75000   
                C              1126.00  1108.85  1112.60 -56.47998   
                C              1201.10  1086.10  1092.25 -17.23999   
...                                ...      ...      ...       ...   
                C                 0.01     0.00     0.08   0.00000   
                C                 0.04     0.00     0.03   0.00000   
                C                 0.01     0.00     0.04  -0.04000   
                C                 0.01     0.00     0.01  -0.04000   
                C                 0.01     0.00     0.01  -0.02000   

                             percentChange  volume  openInterest  \
expiration_date option_type                                        
2020-11-06      C                -5.160956    15.0           4.0   
                C                -4.672994     1.0           2.0   
                C                -5.267798     1.0           2.0   
                C                -4.776401     1.0           3.0   
                C                -1.415039     2.0           2.0   
...                                    ...     ...           ...   
                C                 0.000000     2.0         239.0   
                C                 0.000000    64.0         121.0   
                C               -79.999990     6.0          73.0   
                C               -79.999990     2.0         229.0   
                C               -66.666670   398.0         433.0   

                             impliedVolatility  inTheMoney contractSize  \
expiration_date option_type                                               
2020-11-06      C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
                C                     0.000010        True      REGULAR   
...                                        ...         ...          ...   
                C                     1.121098       False      REGULAR   
                C                     1.070317       False      REGULAR   
                C                     1.117192       False      REGULAR   
                C                     1.046880       False      REGULAR   
                C                     1.062505       False      REGULAR   

                            currency  
expiration_date option_type           
2020-11-06      C                USD  
                C                USD  
                C                USD  
             

### We can use the ```groupby``` method to apply a function such as mean,std,min,max to our data grouped in a particular way ```dataframe.groupby([column1,column2,...]).function()```, where the function operations are applied columnwise

### Note that groupby tipically returns a multi-index dataframe

In [167]:
option_data.groupby(["expiration_date","option_type"]).max() #gives the maximum value for each expiration/option_type combination

contractSymbol        lastTradeDate  strike  \
expiration_date option_type                                                     
2020-11-06      C            AMZN201106C04850000  2020-11-02 19:11:10  4850.0   
                P            AMZN201106P04850000  2020-11-02 19:11:17  4850.0   
2020-11-13      C            AMZN201113C04850000  2020-11-02 19:11:10  4850.0   
                P            AMZN201113P04650000  2020-11-02 19:11:15  4650.0   
2020-11-20      C            AMZN201120C05300000  2020-11-02 19:10:24  5300.0   
                P            AMZN201120P05300000  2020-11-02 19:10:56  5300.0   
2020-11-27      C            AMZN201127C04850000  2020-11-02 19:10:50  4850.0   
                P            AMZN201127P04550000  2020-11-02 19:10:56  4550.0   
2020-12-04      C            AMZN201204C03800000  2020-11-02 19:09:31  3800.0   
                P            AMZN201204P03440000  2020-11-02 19:06:10  3440.0   
2020-12-11      C            AMZN201211C03300000  2020-11-02 19:06:56  3300.0   
                P            AMZN201211P03300000  2020-11-02 19:06:01  3300.0   
2020-12-18      C            AMZN201218C05300000  2020-11-02 19:09:59  5300.0   
                P            AMZN201218P05300000  2020-11-02 19:10:15  5300.0   
2021-01-15      C            AMZN210115C05300000  2020-11-02 19:09:43  5300.0   
                P            AMZN210115P05300000  2020-11-02 19:09:28  5300.0   
2021-02-19      C            AMZN210219C05300000  2020-11-02 19:09:55  5300.0   
                P            AMZN210219P05000000  2020-11-02 19:09:25  5000.0   
2021-03-19      C            AMZN210319C05300000  2020-11-02 19:07:23  5300.0   
                P            AMZN210319P05200000  2020-11-02 19:10:45  5200.0   
2021-04-16      C            AMZN210416C05300000  2020-11-02 18:59:31  5300.0   
                P            AMZN210416P05300000  2020-11-02 19:06:10  5300.0   
2021-06-18      C            AMZN210618C05300000  2020-11-02 19:10:53  5300.0   
                P            AMZN210618P05250000  2020-11-02 19:10:24  5250.0   
2021-07-16      C            AMZN210716C05300000  2020-11-02 18:54:24  5300.0   
                P            AMZN210716P05200000  2020-11-02 18:53:52  5200.0   
2021-08-20      C            AMZN210820C05300000  2020-11-02 19:09:31  5300.0   
                P            AMZN210820P05250000  2020-11-02 18:53:18  5250.0   
2021-09-17      C            AMZN210917C05100000  2020-11-02 19:08:53  5100.0   
                P            AMZN210917P04250000  2020-11-02 19:08:53  4250.0   
2021-10-15      C            AMZN211015C05000000  2020-11-02 19:02:59  5000.0   
                P            AMZN211015P03500000  2020-11-02 18:50:10  3500.0   
2022-01-21      C            AMZN220121C05300000  2020-11-02 19:10:30  5300.0   
                P            AMZN220121P05300000  2020-11-02 19:07:17  5300.0   
2022-06-17      C            AMZN220617C05300000  2020-11-02 19:04:09  5300.0   
                P            AMZN220617P05000000  2020-11-02 16:59:03  5000.0   
2023-01-20      C            AMZN230120C05100000  2020-11-02 19:10:39  5100.0   
                P            AMZN230120P05000000  2020-11-02 19:08:37  5000.0   

                             lastPrice      bid      ask      change  \
expiration_date option_type                                            
2020-11-06      C              1202.70  1139.35  1143.20   42.630005   
                P              1808.05  1887.65  1891.85  246.449950   
2020-11-13      C              1342.65  1068.65  1074.90   22.070007   
                P              1554.05  1684.90  1693.05  191.490020   
2020-11-20      C              1890.27  1759.05  1763.30   15.250000   
                P              2200.95  2342.50  2346.65  381.300050   
2020-11-27      C              1316.60  1096.70  1101.90    6.730003   
                P              1394.30  1590.10  1597.65  105.500000   
2020-12-04      C               957.51   761.60   770.75    0.000000   
 

In [168]:
grouped_option_chain=option_data.groupby(["expiration_date","option_type","strike"]).sum()

In [169]:
grouped_option_chain

contractSymbol        lastTradeDate  \
expiration_date option_type strike                                             
2020-11-06      C           1820.0  AMZN201106C01820000  2020-11-02 18:28:21   
                            1830.0  AMZN201106C01830000  2020-11-02 18:28:21   
                            1840.0  AMZN201106C01840000  2020-11-02 18:28:21   
                            1850.0  AMZN201106C01850000  2020-11-02 18:28:21   
                            1870.0  AMZN201106C01870000  2020-11-02 14:34:36   
...                                                 ...                  ...   
2023-01-20      P           3600.0  AMZN230120P03600000  2020-10-28 16:43:01   
                            3900.0  AMZN230120P03900000  2020-09-30 15:21:16   
                            4000.0  AMZN230120P04000000  2020-10-21 13:47:58   
                            4800.0  AMZN230120P04800000  2020-10-14 19:58:31   
                            5000.0  AMZN230120P05000000  2020-10-22 18:55:34   

                                    lastPrice      bid      ask    change  \
expiration_date option_type strike                                          
2020-11-06      C           1820.0    1148.70  1139.35  1143.20 -62.51001   
                            1830.0    1146.25  1123.35  1127.75 -56.18994   
                            1840.0    1128.45  1116.55  1120.70 -62.75000   
                            1850.0    1126.00  1108.85  1112.60 -56.47998   
                            1870.0    1201.10  1086.10  1092.25 -17.23999   
...                                       ...      ...      ...       ...   
2023-01-20      P           3600.0     980.00  1061.70  1078.45   0.00000   
                            3900.0    1166.30  1227.55  1245.00   0.00000   
                            4000.0    1247.25  1359.95  1376.50   0.00000   
                            4800.0    1795.00  2017.40  2033.00   0.00000   
                            5000.0    2051.20  2193.50  2203.00   0.00000   

                                    percentChange  volume  openInterest  \
expiration_date option_type strike                                        
2020-11-06      C           1820.0      -5.160956    15.0           4.0   
                            1830.0      -4.672994     1.0           2.0   
                            1840.0      -5.267798     1.0           2.0   
                            1850.0      -4.776401     1.0           3.0   
                            1870.0      -1.415039     2.0           2.0   
...                                           ...     ...           ...   
2023-01-20      P           3600.0       0.000000     1.0           2.0   
                            3900.0       0.000000     1.0           0.0   
                            4000.0       0.000000    55.0          59.0   
                            4800.0       0.000000     5.0           4.0   
                            5000.0       0.000000     1.0           1.0   

                                    impliedVolatility  inTheMoney  \
expiration_date option_type strike                                  
2020-11-06      C           1820.0           0.000010           1   
                            1830.0           0.000010           1   
                            1840.0           0.000010           1   
                            1850.0           0.000010           1   
                            1870.0           0.000010           1   
...                                               ...         ...   
2023-01-20      P           3600.0           0.375985           1   
                            3900.0           0.340918           1   
                            4000.0           0.371846           1   
                            4800.0           0.363714           1   
                            5000.0           0.358921           1   

                                   contractSize currency  
expiration_date option_type strike                        
2020-1

In [170]:
grouped_option_chain.loc[('2020-11-06','C'),:]

,contractSymbol,lastTradeDate,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
strike,,,,,,,,,,,,,
1820.0,AMZN201106C01820000,2020-11-02 18:28:21,1148.70,1139.35,1143.20,-62.51001,-5.160956,15.0,4.0,0.000010,1,REGULAR,USD
1830.0,AMZN201106C01830000,2020-11-02 18:28:21,1146.25,1123.35,1127.75,-56.18994,-4.672994,1.0,2.0,0.000010,1,REGULAR,USD
1840.0,AMZN201106C01840000,2020-11-02 18:28:21,1128.45,1116.55,1120.70,-62.75000,-5.267798,1.0,2.0,0.000010,1,REGULAR,USD
1850.0,AMZN201106C01850000,2020-11-02 18:28:21,1126.00,1108.85,1112.60,-56.47998,-4.776401,1.0,3.0,0.000010,1,REGULAR,USD
1870.0,AMZN201106C01870000,2020-11-02 14:34:36,1201.10,1086.10,1092.25,-17.23999,-1.415039,2.0,2.0,0.000010,1,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4650.0,AMZN201106C04650000,2020-11-02 15:54:50,0.01,0.00,0.08,0.00000,0.000000,2.0,239.0,1.121098,0,REGULAR,USD
4700.0,AMZN201106C04700000,2020-10-30 19:55:10,0.04,0.00,0.03,0.00000,0.000000,64.0,121.0,1.070317,0,REGULAR,USD
4750.0,AMZN201106C04750000,2020-11-02 17:32:06,0.01,0.00,0.04,-0.04000,-79.999990,6.0,73.0,1.117192,0,REGULAR,USD


# Some quick final remarks on ```dropna``` and ```interpolate```


In [171]:
### Let's introduce some missing values and see how we can remove or interpolate values

In [172]:
AAPL_copy=AAPL.iloc[:,:-1]
AAPL_copy.iloc[2,:]=np.nan
AAPL_copy.head()

,Adj Close,Close,High,Low,Open
0,115.651062,118.690002,119.199997,116.129997,118.320000
1,113.341728,116.320000,121.989998,116.050003,120.500000
2,NaN,NaN,NaN,NaN,NaN
3,116.430588,119.489998,119.629997,116.440002,117.190002
4,116.157745,119.209999,120.529999,118.570000,119.620003


In [173]:
# Dropna just removes the rows that contain a NaN
AAPL_copy.dropna(axis=0)
# We can also use axis=1 to drop columns

,Adj Close,Close,High,Low,Open
0,115.651062,118.690002,119.199997,116.129997,118.320000
1,113.341728,116.320000,121.989998,116.050003,120.500000
3,116.430588,119.489998,119.629997,116.440002,117.190002
4,116.157745,119.209999,120.529999,118.570000,119.620003
5,116.206482,119.260002,119.669998,117.870003,119.440002
...,...,...,...,...,...
1250,271.399994,271.399994,274.140015,268.480011,271.989990
1251,270.369995,270.369995,277.320007,269.160004,276.989990
1252,269.049988,269.049988,270.850006,266.250000,270.420013
1253,270.040009,270.040009,271.489990,267.619995,268.329987


In [174]:
AAPL_copy.interpolate(method='linear', axis=0)

,Adj Close,Close,High,Low,Open
0,115.651062,118.690002,119.199997,116.129997,118.320000
1,113.341728,116.320000,121.989998,116.050003,120.500000
2,114.886158,117.904999,120.809998,116.245003,118.845001
3,116.430588,119.489998,119.629997,116.440002,117.190002
4,116.157745,119.209999,120.529999,118.570000,119.620003
...,...,...,...,...,...
1251,270.369995,270.369995,277.320007,269.160004,276.989990
1252,269.049988,269.049988,270.850006,266.250000,270.420013
1253,270.040009,270.040009,271.489990,267.619995,268.329987
1254,270.140015,270.140015,271.700012,266.929993,268.589996
